# Project 1 - Building the Backend of an AirBnB Website

The growth of the homesharing and short-term rental markets has presented opportunities and challenges for communities globally. While for some it encourages tourism and provides additional income streams, for others it exacerbates affordable housing shortages. You decide you can create a website sharing the actual data. In this project you will build the "backend" for that website. The following tasks will guide you to experiment with the optimal data structures to store information about listings and their reviews, consider the algorithmic choices you might make to traverse those data structures and even explore the underlying data types you will store the data as. Your goal is to make all of the programs as efficient as possible!

In this project, you will be given a handful of functionalities you are expected to implement for the backend of the website. For each functionality you will use, upgrade or build a data structure to store the data you need for that functionality and determine how to traverse that data efficiently to serve information on your website.

**General note**: You may notice there are several code cells with the tag `excluded_from_script`. These code cells will not be run by the autograder, and you should make sure to preserve these tags.

We start by implementing relevant packages. Note that you do not need to use any NumPy, Pandas or Sklearn functions in this project; they are only imported to set up the dataset and local test cases.

In [2]:
import random, string, hashlib, re, collections
import numpy as np
import pandas as pd
from predict_rating import predict
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from datetime import datetime
import shutil
from collections import deque

/Users/tanvimurke/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


To start, run the following cell to download the data file. Note that this only needs to be run once.

In [3]:
import requests
url = 'http://clouddatascience.blob.core.windows.net/s21-foundation-data-science/systems_data_structures/test_cities.csv'
r = requests.get(url)
with open('test_cities.csv', 'wb') as f:
    f.write(r.content)

In [4]:
airbnb_data = pd.read_csv('test_cities.csv')

/var/folders/gz/48d4h_554811pcq9nyp287340000gn/T/ipykernel_91746/1284643275.py:1: DtypeWarning: Columns (0,1,3,22,28,29,41,45,46,54,61,68) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_data = pd.read_csv('test_cities.csv')


You may see a `DTypeWarning` from the above cell, but you don't need to worry about it.

## Question 1: Analyzing runtime performance and memory requirements of different datatypes (5 points)

One functionality that we want is the ability to predict a listing's price based on its features (e.g., locations, amenities). In addition, this feature should be available on the user's phones, which have limited memory and processing capabilities. We have the option to store our data and model in one of three types: 16-bit floating point, 32-bit floating point, or 64-bit floating point. We also have three criteria to evaluate each option:

1. How much memory does it take to store the model?
1. How long does it take to perform model prediction?
1. How good is the model's prediction?

To answer these questions, let's try out each of our datatype option on a synthetic dataset. Here we assume that a linear regression model is used to carry out the prediction.

Run the code cell below; while you don't need to know the specifics of the NumPy operations that are used, make sure you understand the high-level role of each function.

In [5]:
def construct_dataset():
    """
    Generate the input matrix data (X) and output label vector (y) for a regression problem
    See https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html for more details
    """
    print("I am now executing construct_dataset")
    data = make_regression(n_samples=10000, random_state=42)
    X, y = data[0], data[1]
    X = np.concatenate((np.ones((len(X),1)), X), axis=1)
    return X, y

def train_linear_regression_model(X, y):
    """
    Train a linear regression model to predict the output label vector y based on the input data matrix X
    The output is a weight vector w such that Xw is closest to y, in terms of Mean Squared Erroor
    """
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

def compute_memory_usage(model):
    """
    Compute the number of bytes needed to store the model vector
    """
    return model.nbytes

def compute_prediction_runtime(X, model, N=1000):
    """
    Record the average time taken to perform prediction, sampled from 1000 runs
    """
    total_time = 0
    for _ in range(N):
        start = datetime.now()
        y = X @ model
        end = datetime.now()
        total_time += (end - start).total_seconds()
    return total_time / N

def compute_prediction_error(X, model, y_true):
    """
    Compute the Mean Squared Error from the vector of predicted labels and the vector of true labels
    """
    y_predicted = X @ model
    return np.mean((y_predicted - y_true)**2)

def evaluate_dtype(dtype):
    """
    Return the memory usage, prediction runtime and prediction error when training a linear regression model with the specified data type
    """
    # train the model on 60% of the data
    X_train, X_test, y_train, y_test = train_test_split(*construct_dataset(), test_size = 0.4)
    model = train_linear_regression_model(X_train, y_train)
    
    # convert the test data and trained model vector to the specified dtype
    X_test, model = X_test.astype(dtype), model.astype(dtype)
    
    # perform evaluation
    memory_usage = compute_memory_usage(model)
    prediction_runtime = compute_prediction_runtime(X_test, model)
    prediction_error = compute_prediction_error(X_test, model, y_test)
    print(f"A model stored in data type {dtype} consumes {memory_usage} bytes, takes {prediction_runtime} seconds to perform prediction on average, and has a prediction error of {prediction_error}")
    return memory_usage, prediction_runtime, prediction_error

Now that the set up has finished, let's begin the evaluation! Run the code cell below, then report your finding in the `evaluate_data_type` function.

In [6]:
evaluate_dtype(np.float16);
evaluate_dtype(np.float32);
evaluate_dtype(np.float64);

I am now executing construct_dataset
A model stored in data type <class 'numpy.float16'> consumes 202 bytes, takes 0.0013350460000000042 seconds to perform prediction on average, and has a prediction error of 0.004490931025845336
I am now executing construct_dataset
A model stored in data type <class 'numpy.float32'> consumes 404 bytes, takes 2.6515000000000023e-05 seconds to perform prediction on average, and has a prediction error of 1.548874415705422e-10
I am now executing construct_dataset
A model stored in data type <class 'numpy.float64'> consumes 808 bytes, takes 0.00013619399999999785 seconds to perform prediction on average, and has a prediction error of 1.5046938421399573e-26


Based on the printouts above, fill in the four variables `fastest`, `slowest`, `least_memory`, `best_test_err` in the `select_data_type` function. In particular,
* Each variable should hold one of the three string values: `"float16"`, `"float32"`, `"float64"`.
* `fastest` reports the datatype that yields the lowest **prediction** time
* `slowest` reports the datatype that yields the highest **prediction** time
* `least_memory` reports the datatype that yields the lowest number of bytes
* `best_test_err` reports the datatype that has the lowest test error

In [10]:
def evaluate_data_type():
    fastest = "float32"
    slowest = "float16"
    least_memory = "float16"
    best_test_err = "float64"
    
    
    return fastest, slowest, least_memory, best_test_err

In [11]:
def test_evaluate_data_type():
    assert all(answer in ["float16", "float32", "float64"] for answer in evaluate_data_type())
    print("All tests passed!")
    
test_evaluate_data_type()

All tests passed!


Now you have a good understanding of the trade-offs between different data types. Here are some other points on this topic to think about:
* Note that in our implementation of `evaluate_dtype`, the data type conversion only takes place after the linear regression model has been trained. The reason is that model training happens on our side, so we are typically not too concerned about memory or runtime constraints. However, once a model has been trained, it will be deployed to the client side, which could be a mobile phone or a smart watch with very limited resources; in this case, choosing the appropriate data type to store the model becomes more important. This technique is called [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization).
* Were you surprised about the data type that led to the slowest inference time? Why may this be the case?

## Question 2: Filtering user ids given blacklist and whitelist

Some AirBnB users exhibit bad behaviors (e.g., they are rude to their hosts) and are banned from the platform. The IDs of these users are stored in the text file `blacklist.txt`. We also have a file `whitelist.txt`, which stores the ids of users that are not banned. If the same user id appears in both `blacklist.txt` and `whitelist.txt`, the white list will take priority, i.e., that user is **not** banned. If the input user ID isn’t in the blacklist and isn’t in the whitelist, it will be mapped to `False` (that user is not banned).

Your task is to implement a filter function that, given an input list of user IDs, maps each ID to the boolean `True` if it belongs to a banned user, and `False` otherwise. Note that this involves two sub-tasks:
1. Selecting appropriate data structures to store the blacklisted and whitelisted user IDs.
1. Perform the mapping based on these data structures.

We provide the starting code for sub-task 1 in the function `read_blacklist_and_whitelist` below, which saves the ids into two Python lists. **You should modify this code to store the data more efficiently, for the purpose of performing sub-task 2**.

In [12]:
def read_blacklist_and_whitelist(blacklist_file, whitelist_file):
    '''
    NOTE: you should change the data structure used for storing the blacklisted and whitelisted IDs

    Reads the blacklist and whitelist from input files and store them into appropriate data structures
    
    args:
        blacklist_file (str) : file path of blacklist, each line is separate id
        whitelist_file (str) : file path of whitelist, each line is separate id
        
    returns: Tuple(blacklist, whitelist)
        blacklist (collections[int]) : a data structure storing all of the blacklisted IDs
        whitelist (collections[int]) : a data structure storing all of the whitelisted IDs
    '''
    blacklist = set()
    whitelist = set()
    
    with open(blacklist_file, "r") as f:
        blacklist = set(int(x.strip()) for x in f.readlines())
    
    with open(whitelist_file, "r") as f:
        whitelist = set(int(x.strip()) for x in f.readlines()) 
    
    return blacklist, whitelist

We now store the returned blacklist and whitelist as global variables, so that they can be accessed in later tasks. If you later change your implementation of `read_blacklist_and_whitelist`, make sure to rerun this cell.

In [13]:
blacklist, whitelist = read_blacklist_and_whitelist("blacklist.txt", "whitelist.txt")

### Question 2.1: Checking if a single user is banned (5 points)
Now let's move on to sub-task 2; we will consider a simple case first. Implement the function `check_single_id` that checks for whether a single user ID is banned or not.

In [18]:
def check_single_id(id_to_check, blacklist, whitelist):
    '''
    Checks whether an input ID is banned, based on the stored blacklist and whiteliist
    
    args:
        id_to_check (int) : the user ID that you need to check
        blacklist (collections[int]) : a data structure storing all of the blacklisted IDs
        whitelist (collections[int]) : a data structure storing all of the whitelisted IDs
    
    returns:
        id_state (bool) : True if id_to_check belongs to a banned user, and False otherwise
    '''
    if id_to_check in whitelist:
        id_state = False
    elif id_to_check in blacklist:
        id_state = True
    else:
        id_state = False
    return id_state
    #raise NotImplementedError("Complete this function!")

In [19]:
def test_check_single_id():
    blacklist, whitelist = read_blacklist_and_whitelist("blacklist.txt", "whitelist.txt")
    assert check_single_id(59735, blacklist, whitelist) == False, "Check that you handle cases when id is in both blacklist and whitelist!"
    assert check_single_id(5935, blacklist, whitelist) == True, "Check that you handle cases when id is in blacklist and not in whitelist!"
    print("All tests passed!")
    
test_check_single_id()

# let's also see how long it takes to run this function
%timeit check_single_id(59735, blacklist, whitelist)

All tests passed!
42.2 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Question 2.2: Filtering list of ids (10 points)
Now we move on to the real filtering task. Implement the function `check_list_ids` which, given an input list of IDs, maps each ID to the boolean `True` if it belongs to a banned user, and `False` otherwise. You can either reuse your implementation of `check_single_id`, or implement this task from scratch. Note that the input list of IDs may have very large size, so make sure you do not perform any redundant or repeated operations.

In [31]:
def check_list_ids(input_ids, blacklist, whitelist):
    '''
    Checks whether each ID in an input list of IDs is banned, based on the stored blacklist and whiteliist
    
    args:
        input_ids (List[int]) : the user ID that you need to check
        blacklist (collections[int]) : a data structure storing all of the blacklisted IDs
        whitelist (collections[int]) : a data structure storing all of the whitelisted IDs
    
    returns:
        List[bool] : a list having the same length as input_ids, where the entry at index i
            is True if input_ids[i] is banned, and False otherwise
    '''
 
    '''
    check_dict = {ids: check_single_id(ids, blacklist, whitelist) for ids in input_ids}
    check_list = list(check_dict.values())
    return check_list
    '''

    return [check_single_id(ids, blacklist, whitelist) for ids in input_ids]
    

    #raise NotImplementedError("Complete this function!")

In [32]:
def test_check_list_ids():
    test_input_ids = [15795, 860, 76820, 54886, 6265, 82386, 37194, 87498, 44131, 60263]
    ref = [False, True, True, False, True, False, False, False, False, False]
    assert check_list_ids(test_input_ids, blacklist, whitelist) == ref
    print("All tests passed!")

test_check_list_ids()

# let's also see how long it takes to run this function
random.seed(42)
test_input_ids = [random.randint(0, 100000) for x in range(1000000)]
%timeit check_list_ids(test_input_ids, blacklist, whitelist)

All tests passed!
57.6 ms ± 3.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The autograder will test your code on an input list of roughly 100000 IDs as well, so make sure everything is optimized!

## Question 3: High performance FIFO storage system (10 points)
Next, you want to *dynamically* display the descriptions of AirBnB homes to the user. Imagine that the user enters a search query to AirBnb and gets an initial collection of search results. Then, the user may choose to alter their search query in some ways (for example, changing the start and end date of their stay), causing some new listings to be inserted to the search results, and some old listings to be removed. Your task is to identify a data structure which can efficiently perform these insertion/removal operations. To simplify the context, we will further assume that, when a removal operation takes place, only the oldest element in the collection is removed (oldest in the sense that it was added to the collection before any other element) -- in other words, your collection of search results behaves in a First In First Out (FIFO) manner.

In the cell below, we provide a starting implementation of the [generator function](https://wiki.python.org/moin/Generators) `Storage`, which takes as input an initial list of AirBnB homes `initial_data`, as well as a data socket `data_socket`. `data_socket` is a Python generator that iteratively yields the next operation which should be performed on `initial_data`:
* If `data_socket` yields the string `"generate"`, the oldest element in `initial_data` should be removed and yielded by your function, unless `initial_data` is currently empty. You can assume the elements in `initial_data` are in the same order that you would need to yield them -- the oldest element is at the beginning of the list.
* If `data_socket` yields any other string `x`, insert `x` to `initial_data`.

The current implementation simply operates directly on the list `initial_data`, which may not be the most efficient approach (recall that removing from the head of a list is costly). Your task is to move the elements of `initial_data` to a more suitable data structure and reimplement the insertion/removal operations accordingly on this new data structure.

**Example:** let's say `initial_data = ["10", "20"]` and `data_socket` yields the following strings: `"30", "generate", "generate", "generate", "generate", "40", "generate", "50"`. Then you should:
1. Add `"30"` to `initial_data`
1. Remove and yield the four oldest elements in `initial_data`, if they exist.
1. Add `"40"` and `"50"` to `initial_data`.
1. Finally, `Storage(initial_data, data_socket)` becomes a generator that yields `"10", "20", "30", "40"` (because `"10"` was removed from `initial_data` first, followed by `"20", "30", "40"`).

In [36]:
def Storage(initial_data, data_socket):
    '''
    NOTE: you should modify this function to be more efficient
    
    Dynamically update the collection of search results based on the generator data_socket and the initial result collection initial_data
    
    args:
        initial_data (List[Object]) - the initial collection of search results
        data_socket (generator [Object|String]) - a generator that yields either some object or the string "generate"
        
    yields:
        Object from initial_data or data_socket
    '''
    fifo_storage = deque(initial_data)
    for item in data_socket():
        if item == "generate":
            if len(fifo_storage) > 0:
                yield fifo_storage.popleft()
        else:
            fifo_storage.append(item)

In [37]:
def test_storage():
    def test_socket():
        for x in [3,"generate","generate","generate","generate",4,"generate",5]:
            yield x

    assert list(Storage([1,2], test_socket)) == [1, 2, 3, 4]
    print("All tests passed!")
    
test_storage()

# let's also see how long it takes to run this function
def data_socket():
    # a sample data_socket that yields "generate" once in every 10 elements
    for i in range(10000):
        if i % 10: 
            yield "generate"
        yield airbnb_data["description"].iloc[i % len(airbnb_data)]
        
%timeit list(Storage(list(airbnb_data["description"].values), data_socket));

All tests passed!
40 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Question 4: Finding neighborhood spans (25 points)

To help users navigate the website conveniently, we want to add hyperlinks to the neighborhoods that are mentioned in the housing descriptions. For example, given the description string `"It's a beautiful house in Salt Lake District and you will enjoy it here!"`, we can identify `Salt Lake District` as a reference to a neighborhood, so we can embed it in a hyperlink. In this way, what the user will see on our website is

> It's a beautiful house in [Salt Lake District](https://en.wikipedia.org/wiki/Salt_Lake_City_School_District) and you will enjoy it here!

and they can click on the hyperlink to know more about Salt Lake District. In our context we will not care about where the hyperlink points to, but we do want to identify the exact span that should be embedded in the link. As you can see from the above example, the hyperlink starts at `Salt` and ends at `District`.

But how do we know which word, or collection of words, corresponds to a neighborhood name? Note that our dataset has a column `host_neighbourhood`, which lists the neighborhood of every AirBnB home:

In [7]:
airbnb_data["host_neighbourhood"]

0              Indische Buurt
1              Grachtengordel
2              Grachtengordel
3         Westelijke Eilanden
4           Amsterdam Centrum
                 ...         
243009                    NaN
243010                    NaN
243011                    NaN
243012               Østerbro
243013                    NaN
Name: host_neighbourhood, Length: 243014, dtype: object

So we can collect all the unique entries from this series and preprocess them to form our neighborhood vocabulary. Your task is to identify a suitable data structure for this vocabulary and construct it in the function `build_neighborhood_vocab` below. Then implement the function `find_spans` which, given a housing description and the neighborhood vocabulary, identifies all the hyperlink spans. Each hyperlink span is a tuple with the format `(start_index, end_index, neighborhood_name)`, where the `start_index` and `end_index` denote the index of the neighborhood name's starting word and ending word within the description string.

As an example, with the input string description `"It's a beautiful house in Salt Lake District and you will enjoy it here!"`, the expected output would be `[(6, 8, "salt lake district")]`. Although `salt` is at index 5 and `district` at index 7 in the description, note that "It's" gets preprocessed to "It" and "s". Thus, `It` and `s` are two separate words in this description. Sentece get's transformed to `"It s a beautiful house in Salt Lake District and you will enjoy it here!"` after preprocessing the description text.

**Notes for `build_neighborhood_vocav`**:
* The neighborhood names need to be preprocessed before they are added to the vocabulary. We have provided the implementation of this preprocessing task for you in the function `preprocess_name`. You should not modify this function.
* We also provide the function `check_valid_name`, which returns `True` if a neighborhood name, after preprocessing, is valid (i.e., it should not be `nan` and should have more than five characters). Only valid neighborhood names should be added to the vocabulary.
* You may find the [trie](https://en.wikipedia.org/wiki/Trie) data structure useful in this task.

**Notes for `find_spans`**:
* There are some text preprocessing tasks to perform on the description string; these have been implemented for you at the start of `find_spans` and should not be modifiied.
* Make sure to find all spans; if there are two spans of the same neighborhood, you need to output both.
* For `start_index` and `end_index`, note that indexing starts at 0 and `end_index` is inclusive. If the identified neighborhood consists of a single word, then `start_index` is equal to `end_index`.
* If two spans are overlapping but have different `start_index`, output both of them. For example, let's say our vocabulary has two neighborhoods: `"Great district"` and `"District of Learning"`. If the input description is `"great district of learning"`, the output of `find_spans` should be `[(0,1,"great district"), (1,3,"district of learning")]`.
* If two spans are overlapping and have the same `start_index` (i.e., one span is contained in another), only output the smaller span. For example, let's say our vocabulary has two neighborhoods: `"District"` and `"District of Learning"`. If the input description is `"District of Learning is great!" `, the output of `find_spans` should be `[(0, 0, "district")]`.

In [340]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.isCompletedWord = False
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
        
    def insert(self, word):
        current = self.root
        for letter in word:
            if letter not in current.children:
                current.children[letter] = TrieNode()
            current = current.children[letter]
        current.isCompletedWord = True
            
    def search(self, word):
        current = self.root
        for letter in word:
            if letter not in current.children:
                return False
            current = current.children[letter]
        return current.isCompletedWord
    
    def startsWith(self, word):
        current = self.root
        for letter in word:
            if letter not in current.children:
                return False
            current = current.children[letter]
        return True   
        
def preprocess_name(name):
    """Perform some text cleaning on the neighborhood name"""
    if str(name) == "nan":
        return None
    
    # convert to lowercase and remove "-"
    name = name.lower().replace("-", " ")
    
    # replace occurences of multiple spaces with a single space
    return re.sub(r"\s+", " ", name)

def check_valid_name(processed_name):
    """Check if a neighborhood name is valid"""
    return processed_name is not None and len(processed_name) > 5

def build_neighborhood_vocab(neighborhoods):
    """
    Construct a vocabulary of processed and valid neighborhood names
    
    args:
        neighborhoods (List[str]) : a list of raw and unique neighborhood names from the dataset
    
    returns:
        collections : a data structure that stores the neighborhood names
    """
    trie = Trie()
    neighbor_set = set()
    for word in neighborhoods:
        processed_name = preprocess_name(word)
        if check_valid_name(processed_name):
            neighbor_set.add(processed_name)
    for neighbor in neighbor_set:
        trie.insert(neighbor)
    return trie
         
    #raise NotImplementedError("Implement this task")
    
def find_spans(description, neighborhood_vocab):
    """
    Identify all spans of neighborhood names in a given string description, based on the given neighborhood vocabulary
    
    args:
        description (str) : a string description of an AirBnB home
        neighborhood_vocab (collections) : the vocabulary of all possible neighborhood names
    """
    # preprocess the description text, do not modify this code
    if str(description) == "nan":
        return []
    description = description.lower()
    for p in string.punctuation:
        description = description.replace(p, " ")
    description = re.sub(r"\s+", " ", description)
       
    spans = []
    desc_words = description.split()
    index = 0 
    neighbor_found = False
    
    while index < len(desc_words):
        if neighborhood_vocab.startsWith(desc_words[index]):
            start_index = index
            while index < len(desc_words): 
                next_word = ' '.join(desc_words[start_index:index+1])
                if not neighborhood_vocab.startsWith(next_word):
                    break
                if neighborhood_vocab.search(next_word):
                    end_index = index
                    neighbor_found = True
                index += 1
            if neighbor_found:
                word_span = ' '.join(desc_words[start_index:end_index+1])
                spans.append((start_index, end_index, word_span))
                neighbor_found = False
            index = start_index + 1
        else:
            index += 1
    return spans
            

    # fill in your implementation here
    #raise NotImplementedError("Implement this task")

In [341]:
def test_find_spans():
    neighborhood_vocab = build_neighborhood_vocab(airbnb_data.host_neighbourhood)
    test = [find_spans(x, neighborhood_vocab) for x in airbnb_data.description.head(10)]
    ref = [[(48, 49, 'indische buurt')], [], [(7, 7, 'jordaan')], [], [], [], [], [(18, 18, 'jordaan')],
           [(143, 143, 'grachtengordel'), (149, 149, 'grachtengordel')], [(66, 67, 'park view')]]
    assert all([x1 == x2 for x1, x2 in zip(test,ref)]), "Check your implementation!"
    print("All tests passed!")

%timeit test_find_spans()

All tests passed!
All tests passed!
All tests passed!
All tests passed!
All tests passed!
All tests passed!
All tests passed!
All tests passed!
134 ms ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Question 5: Depth-first search (10 points)
In this question, we will consider the referral feature: user `A` can refer a friend `B` to AirBnB and, if `B` joins, both `A` and `B` get some AirBnB credit. Assume we have a database of referrals, organized in the form of a poly-tree, where each node corresponds to a user. Node `A` is the parent of node `B` (i.e., there is a directed edge from `A` to `B`) if `B` was referred by `A`. Each node can have at most one parent, and some nodes may not have any children.

Given a user ID, we want to identify the chain of referrals that led to this user joining the app. For this purpose, we will use depth-first search (DFS):
1. Visit the first unvisited child node of a current node; in this case we would define the "first" child as the leftmost child.
1. If all nodes were visited or if a node is a leaf (has no child nodes), go up.
1. As soon as you find the target node, stop iterating.

For example, given the following poly-tree:

![](polytree.png)

DFS would traverse the nodes in the following order: `1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11`. If the target node is `9`, we return the chain of ancestors of `9` as `[6, 7, 8, 9]` (note that `9` itself is also included in this list). In our context, we represent the above tree as a nested dictionary, where each node is a key that maps to all of its subtrees:

```
tree = {
  '1 : {'2' : {}},
  '3' : {'4' : {}, '5' : {}},
  '6' : {'7' : {'8': {'9' : {}, '10' : {}}}},
  '11' : {} 
}
```

Implement the function `dfs` that, given a target user ID and the referral database, computes (1) the chain of ancestors of the target node, and (2) the full DFS traversal sequence leading to the target node.

**Notes**:
* The target user ID may not be present in the dataset -- for this edge case, the chain of parents is an empty list, and the DFS traversal sequence should contain every node in the tree.
* Each node should only appear at most once in the returned DFS traversal sequence, as well as the chain of parents.

In [344]:
def dfs(target, tree):
    '''
    Find the chain of referrals leading to a target user ID
    
    args:
        target (String) : the target user id
        tree (dict) - the referral poly-tree represented in nested dictionary format

    returns:
        chain_of_ancestors (List[String]) - chain of parents of the target node, or empty list if the target node has no parent
        history (List[String]) - the full DFS traversal sequence leading to the target node
    '''
    def dfs_recursive(target, tree, history, chain_of_ancestors):
        for key, child_node in tree.items():
            history.append(key)

            if key == target:
                chain_of_ancestors.append(key)
                return True

            result = dfs_recursive(target, child_node, history, chain_of_ancestors)

            if result:
                chain_of_ancestors.append(key)
                return True

        return False 
    
    history = []
    chain_of_ancestors = []
    dfs_recursive(target, tree, history, chain_of_ancestors)
    chain_of_ancestors.reverse()
    return chain_of_ancestors, history 
  
   
    #raise NotImplementedError("Complete this function!")

In [345]:
def test_dfs():
    tree = {
      '1' : {'2':{}},
      '3' : {'4':{}, '5':{}},
      '6' : {'7': {'8': {'9':{}, '10': {}}}},
      '11' : {} 
    }
    
    target = '9'
    parents_ref = ['6', '7', '8', '9']
    history_ref = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    assert dfs(target, tree) == (parents_ref, history_ref), "Check you implementation for when node is in a tree!"
    
    target = '111'
    parents_ref = []
    history_ref = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
    
    assert dfs(target, tree) == (parents_ref, history_ref), "Check you implementation for when node can not be found!"
    print("All tests passed!")

test_dfs()


All tests passed!


## Question 6: Breadth-first search (10 points)
Now we are interested in counting the number of users at each "level" of the tree -- e.g., how many users joined on their own, how many users had one ancestor, how many users had two ancestors. For this purpose, we will use breadth-first search (BFS):
1. Set current depth `k` to `0`.
1. Get all nodes that are maximum depth `k` from the root node.
1. Increase `k`.

As an example, consider the same tree from Question 5:
![](polytree.png)

In this case, the BFS traversal would yield `[['1', '3', '6', '11'], ['2', '4', '5', '7'], ['8'], ['9', '10']]`. Note that this is a nested list, and every inner list is a collection of nodes at a certain level in the tree. Implement the function `bfs` that, given a referral database, returns the list of nodes at each level.

**Notes**:
* The order of the nodes within each inner list is not important; you can order them however you like.


In [342]:
def bfs(tree):
    '''
    Traverses a tree with Breadth-First Search algorithm.
    Args:
        tree (Nested Dictionary) - tree to be traversed. 
                    Key:value relationship indicates parent:child nodes relationship.
                    Empty set indicates leaf node.
    Returns:
        levels (List[List[String]]) - full traversal history. Each sublist is one full level with correct order.
    '''

    result = []
    queue = deque()
  
    queue.append(tree)
    
    while queue:
        size = len(queue)
        inner_list = []
        for i in range(size):
            subtree =  queue.popleft()

            for parent, childnode in subtree.items():
                inner_list.append(parent)
                
                if childnode:
                    queue.append(childnode)

        result.append(inner_list)

    return result

    #raise NotImplementedError("Complete this function!")

In [343]:
def test_bfs():

    tree = {
      '1' : {'2':{}},
      '3' : {'4':{}, '5':{}},
      '6' : {'7': {'8': {'9':{}, '10': {}}}},
      '11' : {} 
    }
    
    levels_ref = [['1', '3', '6', '11'], ['2', '4', '5', '7'], ['8'], ['9', '10']]
    levels_output = bfs(tree)
    assert len(levels_ref) == len(levels_output)
    for i in range(len(levels_ref)):
        assert sorted(levels_ref[i]) == sorted(levels_output[i])

    print("All tests passed!")

test_bfs()

All tests passed!


## Memory hierarchy and caching

Now you have decided to add a bit of magic (or machine learning!) to your project and come up with a model that, given a user review text, yields the predicted rating for that review. You have decided to use this model on a stream of incoming data - namely, whenever you get some listing (which might be an existing or a new listing), predict the user rating for it.
  
Unfortunately, it appears that your magical model is processing each text rather slowly and while it does not matter for rare listings, it does matter for listings that appear often. One possible workaround is to store some of the previously computed ratings, so that they do not need to be computed again. By analyzing the resources on the expected devices, you found that you can have up to two storages - a "hot" storage, fast but with limited memory, and a "cold" one, slower but with more memory available.

### Question 7.1: Exploratory performance analysis (10 points)

Below is how your magical model can be used: it accepts a string and outputs an integer value as predicted rating. In the scope of this project, you do not need to know how the rating is calculated.

In [228]:
test_string = 'This course is great and I feel like I am learning a lot!'
predicted_rating = predict(test_string)
print(predicted_rating)

9


Below is one way to create a hot storage and a cold storage.
* The hot storage is a dictionary in RAM that simply maps each review text to its predicted rating. Then, given a review text, we can retrieve its predicted rating by getting the value which that text maps to in our dictionary.
* The cold storage is a text file where each line stores a pair of hashed review text and predicted rating. This text file has more storage capabilities than what is available in RAM, but retrieving a review's rating takes longer because we need to loop through the file line by line. Note again that the review texts are hashed before being saved to the file, as a simple way of compressing data and saving some storage.

In [229]:
# this is the hash function that is applied to every review text in the cold storage
def hash_str(s):
    return hashlib.sha1(s.encode("utf-8")).hexdigest()

In [230]:
class HotStorage:
    def __init__(self):
        self.cache_ram = {}
    
    def save_rating(self, review_text, predicted_rating):
        self.cache_ram[review_text] = predicted_rating
    
    def get_rating(self, review_text):
        return self.cache_ram.get(review_text)

class ColdStorage:
    def __init__(self, filename = "cache_disk.txt"):
        self.filename = filename
    
    def save_rating(self, review_text, predicted_rating):
        hashed_text = hash_str(review_text)
        with open(self.filename, "a+") as f:
            f.write(hashed_text + " " + str(predicted_rating) + "\n")
    
    def get_rating(self, review_text):
        hashed_text = hash_str(review_text)
        with open(self.filename, "r") as f:
            for line in f:
                review_rating_pair = line.replace('\n', '').split(' ')
                if review_rating_pair[0] == hashed_text:
                    return int(review_rating_pair[1])

Here is how we can save and retrieve a predicted rating from the two storages implemented above:

In [231]:
hot_storage, cold_storage = HotStorage(), ColdStorage()
hot_storage.save_rating(test_string, predicted_rating)
cold_storage.save_rating(test_string, predicted_rating)

print(hot_storage.get_rating(test_string))
print(cold_storage.get_rating(test_string))

9
9


Now that the set up is done, let's move on to evaluation. Assume the review text `test_str` comes up again in our stream of data, there are now three ways we can get its predicted rating:
1. Use the `predict` function to recompute the predicted rating.
1. Retrieve the predicted rating from the hot storage.
1. Retrieve the predicted rating from the cold storage.

Now it's your turn to write some code that implements each of the three options above and compares their runtime. The IPython magic command `%timeit` can be very useful for this task. Also make sure to tag every cell that has your experimental code with the tag `excluded_from_script`, so that they are ignored by the autograder; otherwise, IPython commands and print statements will interfere with the grading.

In [232]:
## your code here

%timeit predict(test_string)
%timeit hot_storage.save_rating(test_string, predicted_rating)
%timeit cold_storage.save_rating(test_string, predicted_rating)

20 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.9 ns ± 0.721 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
28.7 µs ± 700 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Based on your observations, replace `None` values in the function below with one of three strings: `"compute"`, `"ram"` or `"disk"`. `"compute"` refers to recomputing the predicted rating; `"ram"` refers to using the hot storage, and `"disk"` refers to using the cold storage. You can assume that the predicted rating is already present in both the hot and the cold storage.

In [233]:
def data_hierarchy_selection():    
    # Which way of getting the predicted rating is the fastest? 
    fastest_retrieval = "ram"
    
    # Which way of getting the predicted rating is the slowest? 
    slowest_retrieval = "compute"
    
    return fastest_retrieval, slowest_retrieval

In [234]:
def test_data_hierarchy_selection():
    fastest_retrieval, slowest_retrieval = data_hierarchy_selection()
    assert fastest_retrieval in ["compute", "ram", "disk"]
    assert slowest_retrieval in ["compute", "ram", "disk"]
    print("All tests passed!")
    
test_data_hierarchy_selection()

All tests passed!


All is well when there is only one predicted rating to retrieve. Unfortunately, AirBnB has a very large number of ratings, which cannot fit into either the hot or cold storage of the user's device. The workaround in this case is to use our available storages more selectively: we will only store values that are likely to come up again later. There is, of course, no way to know whether a value will come up again later, but a popular heuristic we can employ is that the more recent values are more likely to appear again and should be prioritized.

To begin implementing this heuristic, first examine the `Cache` class below, which manages both a hot storage and a cold storage simultaneously. Its functionality is mostly taken from the classes `HotStorage` and `ColdStorage` from the previous question, but we also have functions that remove data from the storage. You do not need to modify anything in `Cache`, but will need to use this class for the next question.

In [235]:
class Cache:
    def __init__(self):
        # make a copy of the initial cache file to be the working cache file
        shutil.copy("cache_disk.txt", "working_cache_disk.txt")
        self.cache_ram = {}
        self.cache_disk = "working_cache_disk.txt"
        with open(self.cache_disk, "w") as f:
            f.write("")
    
    def get_from_ram_cache(self, text):
        '''
        Get value from RAM cache. 
        Args:
            text - raw text for which predicted value is needed
        Return:
            Integer or None - predicted rating for text or None, 
                                if there requested text is not stored in this cache
        '''
        return self.cache_ram.get(text)
    
    def add_to_ram_cache(self, text, value):
        '''
        Add value to RAM cache. 
        Args:
            text [String] - raw text
            rating [Integer] - rating that should be saved
        '''
        self.cache_ram[text] = value
    
    def remove_from_ram_cache(self, text):
        '''
        Remove value from RAM cache. 
        Args:
            text [String] - raw text
        '''
        del self.cache_ram[text]
    
    def ram_size(self):
        '''
        Get size of RAM cache. 
        Return:
            Integer - current number of elements in RAM cache.
        '''
        return len(self.cache_ram)
    
    def get_from_disk_cache(self, text):
        '''
        Get value from disk cache. 
        Args:
            text [String] - raw text for which predicted value is needed
        Return:
            Integer or None - predicted rating for text or None, 
                                if there requested text is not stored in this cache
        '''
        hashed_text = hash_str(text)
        with open(self.cache_disk, "r") as f:
            for line in f:
                review_rating_pair = line.replace('\n', '').split(' ')
                if review_rating_pair[0] == hashed_text:
                    return int(review_rating_pair[1])

    def add_to_disk_cache(self, text, rating):
        '''
        Add value to disk cache. 
        Args:
            text [String] - raw text
            rating [Integer] - rating that should be saved
        '''
        key = hash_str(text)
        with open(self.cache_disk, "a+") as f:
            f.write(key + " " + str(rating) + "\n")
            
    def remove_from_disk_cache(self, text):
        '''
        Remove value from disk cache. 
        Args:
            text [String] - raw text
        '''
        hashed_text = hash_str(text)
        with open(self.cache_disk, "r") as f:
            lines = f.readlines()
        with open(self.cache_disk, "w") as f:
            for line in lines:
                review_rating_pair = line.replace('\n', '').split(' ')
                if review_rating_pair[0] != hashed_text:
                    f.write(line)
     
    def disk_size(self):
        '''
        Get size of disk cache. 
        Return:
            Integer - current number of elements in disk cache.
        '''
        i = -1
        with open(self.cache_disk) as f:
            for i, l in enumerate(f):
                pass
        return i + 1
    
    def get(self, text):
        '''
        Compute the predicted rating by using the magical model
        Args:
            text  [String] - raw text
        Return:
            Integer - rating
        '''
        return predict(text)

### Question 7.2: "Most recent" caching strategy (10 points)

Let's implement the form of caching that is based on temporal locality. We first denote `ram_max` and `disk_max` as the maximum number of elements that can be stored in RAM (hot storage) and in disk (cold storage) respectively. Here each element is a pair of review text and predicted rating. Given a new review text, which we denote as `new_text`, the caching algorithm works as follows:

```
If new_text is in the RAM cache, return its stored predicted rating, and mark this pair of (new_text, stored predicted rating) as the most recent element in RAM.
If the hashed value of new_text is in the disk cache, return its stored predicted rating, and mark this pair of (hashed_new_text, stored predicted rating) as the most recent element in disk.
If the above cases do not apply, compute the predicted rating of new_text, which we denote as predicted_rating. Then:
    If the RAM cache's size is equal to ram_max:
        If the disk cache's size is equal to disk_max, remove the least recent element from the disk cache.
        Move the least recent element in the RAM cache to the disk cache (i.e., remove it from RAM cache and add it to disk cache).
    Store the pair (text, predicted_value) in RAM cache
    Update the most recent element in ram / disk and return predicted_rating 
```

**Notes**:
* Note that `MostRecentCache` inherits from `Cache`, so you are able to use or overwrite the methods in `Cache`. You are also free to modify the `Cache` class above, although doing so is not required.
* You may need to initialize additional fields in the constructor to hold metadata about the RAM cache and disk cache.

In [238]:
class MostRecentCache(Cache):
    def __init__(self, ram_max, disk_max):
        '''
        Args:
            ram_max (Integer) - max number of elements that at any moment of time should be in hot storage
            disk_max (Integer) - max number of elements that at any moment of time should be in cold storage
        '''
        super().__init__()
        self.ram_max = ram_max
        self.disk_max = disk_max
        self.recent_stack_ram = deque()
        self.recent_stack_disk = deque() 
        
        # initialize additional fields here
        
    def get(self, new_text):
 
        ram_new_rating = self.get_from_ram_cache(new_text)
        if ram_new_rating is not None:
            self.recent_stack_ram.append(new_text)
            return ram_new_rating
        
        disk_new_rating = self.get_from_disk_cache(new_text)
        if disk_new_rating is not None:
            self.recent_stack_disk.append(new_text)
            return disk_new_rating
            
        compute_new_rating = super().get(new_text)
        
        if self.ram_size() >= self.ram_max:
            if self.disk_size() >= self.disk_max:
                recent_disk = self.recent_stack_disk.popleft()
                self.remove_from_disk_cache(recent_disk)
 
            recent_ram = self.recent_stack_ram.popleft()
            move_rating = self.get_from_ram_cache(recent_ram)
            self.add_to_disk_cache(recent_ram,move_rating)
            self.recent_stack_disk.append(recent_ram)
            self.remove_from_ram_cache(recent_ram)
            
        self.add_to_ram_cache(new_text, compute_new_rating)
        self.recent_stack_ram.append(new_text)
        return compute_new_rating
 
    
        # implement the caching algorithm
        #raise NotImplementedError("Complete this function!")

In [239]:
def test_most_recent_cache():
    c = MostRecentCache(5, 20)

    for i in range(200):
        c.get('test' + str(i))
        
    assert list(c.cache_ram.keys()) == ['test195', 'test196', 'test197', 'test198', 'test199'], "RAM cache incorrect!"
    
    keys_disk = set()
    with open(c.cache_disk, "r") as f:
        for line in f:
            li = line.replace('\n', '').split(' ')
            keys_disk.add(li[0])
    
    assert keys_disk == {'04a81612669a6503309a3d0a7eb144d47d2822db',
                         '06587144b7eefa0a96389e4edc25db495a24a2cc',
                         '0e51c498407d2bf5bf19a6be003cc65e194337a8',
                         '17dd06e1d4e670fb47b41a753da6a6dfcd87bf30',
                         '26722c1c23e2ab91a5668f711753be9d8a1f6992',
                         '2a373681c4cd01ca424fcc555dd834e417439ce9',
                         '34d422df9f806e415383af86c4400ef7900c4b8f',
                         '5fe02a47e76b0fd9c9297aa1f550df9465990d8b',
                         '6656f767a33b26804aca1f4095ee04b46f9f83dc',
                         '6fbd48d13c9d5e5e4534aec0aca98fd5efecca61',
                         '899ec74db661e24fe58a4f6f302283acd4935ccf',
                         '8e3356790f84402bda95cca9b46fe4b8d8d01e3f',
                         'a15f59d3eb58f1029ccb0a2646d24a3c8a917c29',
                         'a19c3ea08bd1aede544ae465d3799f46d31dd9b7',
                         'ab0fc2097f253a95cc4d689474eb1d7774fa9bb7',
                         'ad1c952a702edca89390aed15dda415100544632',
                         'c34af7f961e92657a743b7390a76e0107c05abc8',
                         'e8efd85f316e5d8594ec24cac041d406ede83fb6',
                         'f8c8ae4bc1e7e9dd003b189787de87c05f413a53',
                         'fed660342e30a9cacdb314c3e8fde3cdab8dba87'}, "Disk cache incorrect!"
    
    assert c.get('test') == 10, "Prediction of new values is incorrect!"
    assert c.get('test196') == 9, "Prediction of values from RAM is incorrect!"
    assert c.get('test178') == 9, "Prediction of values from disk is incorrect!"
    print("All tests passed!")

test_most_recent_cache()

All tests passed!
